In [ ]:
pip install ultralytics opencv-python numpy torch


In [ ]:
import os
os.chdir('/content')

current_directory = os.getcwd()
print(current_directory)


/content


In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 Pose Model (for keypoints)
model = YOLO("yolov8n-pose.pt")

# Load the video
video_path = "/content/sitting or standing posture_vedio.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
output_path = "/mnt/data/Cow_Posture_Detection.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Function to classify posture based on keypoints
def classify_posture(keypoints):
    """
    Determines if a cow is standing or sitting based on keypoints.
    - If leg keypoints are close to the ground -> Sitting
    - If legs are extended and back is high -> Standing
    """
    try:
        # Check if keypoints are valid
        if len(keypoints) < 13:
            return "Unknown"

        back_y = keypoints[8][1]  # Back keypoint
        leg1_y = keypoints[10][1]  # Front leg
        leg2_y = keypoints[12][1]  # Back leg

        # If legs are at similar height as back, the cow is sitting
        if abs(back_y - leg1_y) < 20 and abs(back_y - leg2_y) < 20:
            return "Sitting"
        else:
            return "Standing"
    except:
        return "Unknown"

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO Pose detection
    results = model(frame)

    standing_count = 0
    sitting_count = 0

    for result in results:
        if result.keypoints is None or len(result.keypoints.xy) == 0:
            continue  # Skip frames where no keypoints are detected

        for keypoints in result.keypoints.xy.cpu().numpy():
            if len(keypoints) < 13:  # Ensure enough keypoints are detected
                continue

            # Draw keypoints
            for x, y in keypoints:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 0), -1)

            # Classify posture
            posture = classify_posture(keypoints)

            if posture == "Standing":
                standing_count += 1
                color = (0, 255, 0)  # Green for standing
            else:
                sitting_count += 1
                color = (0, 0, 255)  # Red for sitting

            # Draw label
            x, y = int(keypoints[8][0]), int(keypoints[8][1])
            cv2.putText(frame, posture, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display count on the video
    cv2.putText(frame, f"Standing: {standing_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Sitting: {sitting_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write frame to output video
    out.write(frame)

cap.release()
out.release()

print("Processing complete! Download the processed video here:")
print(output_path)



0: 384x640 (no detections), 198.6ms
Speed: 6.8ms preprocess, 198.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 197.3ms
Speed: 4.3ms preprocess, 197.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 197.7ms
Speed: 3.6ms preprocess, 197.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 206.2ms
Speed: 3.5ms preprocess, 206.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 187.2ms
Speed: 4.0ms preprocess, 187.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 207.8ms
Speed: 4.1ms preprocess, 207.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 179.8ms
Speed: 4.1ms preprocess, 179.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 180.2ms
Speed: 3.7ms prepr

In [ ]:
from IPython.display import display, HTML

video_path = "/content/sitting or standing posture_vedio.mp4"
display(HTML(f"""
    <video width="640" height="480" controls>
        <source src="{video_path}" type="video/mp4">
    </video>
"""))


In [ ]:
import os
print(os.path.exists("/mnt/data/Cow_Posture_Detection.mp4"))


False


In [ ]:
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("No more frames to process.")
        break

    frame_count += 1
    print(f"Processing frame {frame_count}...")

    results = model(frame)
    print(f"Detected objects: {len(results)}")

    out.write(frame)  # Ensure this line executes

cap.release()
out.release()
print(f"Total frames processed: {frame_count}")


Total frames processed: 0


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("/mnt/data/Cow_Posture_Detection.avi", fourcc, fps, (width, height))


In [ ]:
from google.colab import files
files.download("/content/sitting or standing posture_vedio.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>